In [13]:
import numpy as np
import hashlib,hmac, time

In [14]:
# chuyen chuoi ki tu thanh cac so hexa
def char_to_hex(char): 
    return hex(ord(char))[2:]

In [15]:
"""
dem xem can bao nhieu so 0 de them vao padding,
padding bit 1 va mot so nhat dinh cac so 0 de xau ki tu s % 512 = 448
"""
def count_0(string):
    res = len(string)*4
    t = 0
    while(res%512 != 448):
        res += 1
        t +=1
    return t

In [16]:
# doi ca xau sang hex
def string_to_hex(string):
    res = ""
    for i in string:
        res += char_to_hex(i)
    return res
# chuyen chuoi "abc" sang ham bam
string_to_hex("abc")

'616263'

In [17]:
"""
chen cac bit 0
"""
def padding_bit(string,hex_or_not = False):
    if(hex_or_not == False):
        tmp = string_to_hex(string)
    else:
        tmp = string
    res = tmp + "8"
    
    num = count_0(res)
    res = res + "0"*int(num/4)   
    #print(k)
    """
    sau khi xau ki tu co do dai s % 512 = 448 thi them vao 64 bit de
    xau ki tu co the chia thanh n block 512 bit hay 64 byte
    """
    last_length_padding = hex(len(tmp)*4)[2:]
    #print(len(last_length_padding))
    return res  + (16-len(last_length_padding))*"0"+last_length_padding

In [18]:
# chuoi "abc" chuyen sang hex va them cac so 0 vao
padding_bit("abc")

'61626380000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018'

In [19]:
"""
tao cac blocks
"""
def create_block(after_padding_string):
    res = []
    block = len(after_padding_string)/128
    for i in range(int(block)):
        tmp_res = []
        tmp = after_padding_string[i*128:(i+1)*128]
        for j in range(16):
            tmp_res.append(np.array(int(tmp[j*8:(j+1)*8:1],16)).astype(np.uint32))
        res.append(tmp_res)
    
    return np.array(res)
   

In [20]:
create_block(padding_bit("abc"))

array([[1633837952,          0,          0,          0,          0,
                 0,          0,          0,          0,          0,
                 0,          0,          0,          0,          0,
                24]], dtype=uint32)

In [21]:
"""
tao ham f
"""
def f(x,y,z,t):
    if(0<=t and t <=19):
        return ( x & y) ^ (~x & z)
    if(40<=t <=59):
        return (x&y)^(y&z)^(z&x)
    else:
        return x^y^z

In [22]:
"""
khoi tao cac bien ban dau
"""
import numpy as np
K = np.array( [0x5a827999,0x6ed9eba1,0x8f1bbcdc,0xca62c1d6]).astype(np.uint32)
H_init = np.array([0x67452301,0xefcdab89,0x98badcfe,0x10325476,0xc3d2e1f0]).astype(np.uint32)



In [23]:
"""
phep dich vong
"""
def left_rotate(n,b):
    return  ((n << b) | (n >> (32 - b))) & 0xffffffff

In [24]:
def sha(blocks):
    H_before_state = np.array([0x67452301,0xefcdab89,0x98badcfe,0x10325476,0xc3d2e1f0]).astype(np.uint32)
    abcde = H_before_state.copy()
    res = abcde.copy()
    #print(blocks.shape)
    for i in range(blocks.shape[0]):

        abcde = H_before_state.copy()
        
        W = np.zeros(80).astype(np.uint32)

        for t in range(80):
            if(t < 16):
                W[t] = blocks[i][t]
            else:
                W[t] = left_rotate(W[t-3]^W[t-8]^W[t-14]^W[t-16],1)
        
            
            T = (left_rotate(abcde[0],5)+f(abcde[1],abcde[2],abcde[3],t)+K[int(t/20)]+abcde[4]+W[t])&0xffffffff
            abcde[4] = abcde[3]
            abcde[3] = abcde[2]
            abcde[2] = left_rotate(abcde[1],30)
            abcde[1] = abcde[0]
            abcde[0] = T
           

        res = abcde + H_before_state
        H_before_state = res.copy()
    
    a =""
    for i in range(5):
        a += "0"*(8-len(hex(res[i])[2:]))+hex(res[i])[2:]
    return a
        

In [26]:
x = sha(create_block(padding_bit("abc",hex_or_not=False)))

In [27]:
x  

'a9993e364706816aba3e25717850c26c9cd0d89d'

In [28]:
password = "qA090296"
pivot = 1607048354.1137462

In [29]:
"""
khai bao ipad va opad
"""
ipad = '36'*64
opad= '5c'*64

In [36]:
def my_hmac(key,mess,key_type_hex = True):
    m = string_to_hex(mess)
    k = 0
    if(key_type_hex == True):
        k = key
    else:
        k = string_to_hex(key)

    
    if(len(k)> 128):
        k = sha(create_block(padding_bit(k,hex_or_not=True)))
    
    if(len(k) < 128):
        k =  k + "0"*(128-len(k))
    
    o_key_pad = int(k,16)^int(opad,16)
    o_key_pad = "0"*(128-len(hex(o_key_pad)[2:])) + hex(o_key_pad)[2:]
    
    i_key_pad = int(k,16)^int(ipad,16)
    i_key_pad = "0"*(128-len(hex(i_key_pad)[2:])) + hex(i_key_pad)[2:]
   
    small_hash = sha(create_block(padding_bit(i_key_pad + m,hex_or_not=True)))

    
    return sha(create_block(padding_bit(o_key_pad+small_hash,hex_or_not = True)))
    
    
    

In [37]:
#kiem tra ham hmac tu tao voi key la "key", mess la "abc"
my_hmac(key = "key", mess="abc", key_type_hex=False)

'4fd0b215276ef12f2b3e4c8ecac2811498b656fc'

In [59]:
# su dung cac thu vien cua python de tao ham sinh otp
import time, hashlib, hmac
def truncate(hmac_res):
    offset = int(hmac_res[38:],16)&0xf
    return int(hmac_res[offset*2:offset*2+8],16)%10**6

password = "qA090296"
pivot = 1607048354.1137462 # thoi gian server dong bo voi client
"""
def truncate(hmac_res):
    offset = int(hmac_res[38:],16)&0xf
    return int(hmac_res[offset*2:offset*2+8],16)%10**6
"""
def otp_generate(mess):
    t = time.time()
    intput_key =  str(int((t-pivot)/30))
    h = hmac.new(key=bytes(intput_key.encode('utf-8')), msg=bytes(mess.encode('utf-8')), digestmod=hashlib.sha1)
    res = str(truncate(h.hexdigest()))
    return '0'*(6-len(res))+ res

In [60]:
otp_generate(password)

'240193'

In [58]:
loop_end = 1
while(True):
    x = input()
    if(x == "lay tao otp"):
        print(otp_generate(password))
    if(x == 'end'):
        break

lay tao otp
240193
end
